In [1]:
import os
import subprocess
import time
import shutil
import pandas as pd
import multiprocessing

# Configuration
n_threads = multiprocessing.cpu_count()
os.environ['OMP_NUM_THREADS'] = str(n_threads)
print(f"Using {n_threads} threads")

# Parameters
n_steps = 20000
r_restarts = 16
loop_iterations = 3 # Run the loop 3 times

def run_tree_packer(iteration):
    print(f"\n--- Iteration {iteration}: Running Tree Packer ---")
    cmd = ["./tree_packer.exe", "-n", str(n_steps), "-r", str(r_restarts)]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    final_score = None
    for line in process.stdout:
        print(line, end='')
        if "Final:" in line:
            try:
                final_score = float(line.split()[1])
            except:
                pass
                
    process.wait()
    
    if process.returncode != 0:
        print("Tree Packer failed.")
        return False
    
    # Check if output file exists (submission_v21.csv based on previous knowledge, or maybe it overwrites submission.csv?)
    # The code seems to output to submission_v21.csv usually, but let's check.
    # Actually, looking at the code dump in previous turn, I didn't see the output filename.
    # But the python wrapper in the reference notebook expected `submission_{version}.csv`.
    # Let's assume it outputs `submission_v21.csv` or similar.
    # I'll check for any new CSVs.
    
    output_file = "submission_v21.csv" # Default in reference
    if os.path.exists(output_file):
        print(f"Found {output_file}. Updating submission.csv")
        shutil.copy(output_file, "submission.csv")
        return True
    elif os.path.exists("submission.csv"):
        # Maybe it updated submission.csv directly?
        print("No submission_v21.csv found. Checking if submission.csv was modified...")
        return True
    else:
        print("No output file found.")
        return False

def run_bp(iteration):
    print(f"\n--- Iteration {iteration}: Running Backward Propagation ---")
    cmd = ["./bp.exe", "submission.csv", "submission_bp.csv"]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    for line in process.stdout:
        print(line, end='')
        
    process.wait()
    
    if process.returncode == 0 and os.path.exists("submission_bp.csv"):
        print("BP finished. Updating submission.csv")
        shutil.copy("submission_bp.csv", "submission.csv")
        return True
    else:
        print("BP failed.")
        return False

# Main Loop
for i in range(loop_iterations):
    print(f"Starting Loop {i+1}/{loop_iterations}")
    
    # Step 1: Tree Packer
    if not run_tree_packer(i+1):
        break
        
    # Step 2: BP
    if not run_bp(i+1):
        break
        
print("Optimization Loop Completed.")

Using 26 threads
Starting Loop 1/3

--- Iteration 1: Running Tree Packer ---


Tree Packer v21 - ENHANCED (26 threads)
NEW: Swap moves, multi-angle restarts, higher SA temperature
Iterations: 20000, Restarts: 16
Processing all n=1..200 concurrently
Loading submission.csv...
Loaded 200 configs
Initial: 139.580195

Phase 1: Parallel optimization...

n=  1: 1.000000 -> 0.661250 (33.8750%)


n=  2: 0.733589 -> 0.450975 (38.5248%)


n=  3: 0.744063 -> 0.434914 (41.5488%)


n=  4: 0.780252 -> 0.439636 (43.6546%)


...

n= 64: 0.596534 -> 0.579226 (from n=65 removal, 2.9014%)


n= 63: 0.607337 -> 0.585556 (from n=64 removal, 3.5862%)


n= 62: 0.613592 -> 0.593854 (from n=63 removal, 3.2168%)


n= 23: 0.550693 -> 0.534285 (from n=24 removal, 2.9795%)
n= 13: 0.536318 -> 0.531099 (from n=14 removal, 0.9731%)
Pass 1 complete, continuing...

Initial: 119.705455
Final:   118.193395
Improve: 1.512061 (1.2632%)
Phase 1 improved: 194 configs
Phase 2 back-prop improved: 39 configs
Time:    831.4s (with 26 threads)
Saved submission_v21.csv
Found submission_v21.csv. Updating submission.csv

--- Iteration 3: Running Backward Propagation ---


Backward Propagation Optimizer
Loading submission.csv...
Loaded 200 configurations
Starting Backward Propagation...
Initial score: 118.19339450

improved 169 from n=170 10.30284106 -> 10.30036792
improved 117 from n=136 8.49386199 -> 8.49070734
improved 116 from n=136 8.48733082 -> 8.47511045
improved 95 from n=96 7.58295624 -> 7.57787915
improved 36 from n=69 4.67075032 -> 4.64942363
improved 36 from n=68 4.64942363 -> 4.64701717


Backward Propagation Complete!
Total improvements: 6
Final score: 118.183895065702
Saving to submission_bp.csv...
Done!
BP finished. Updating submission.csv
Optimization Loop Completed.
